## Notebook for the simulation of a magnetic Tetrahedron - no boundary layer

In [ ]:
import os
# Please edit this line to provide the to the root directory of your copy of MPT-Calculator
Your_Path='/Users/cgledger/Coding/MPT-Calculator_v2/MPT-Calculator'
#Your_Path='/home/james/Desktop/Updated_MPT_Calculator/MPT-Calculator'
os.chdir(Your_Path)
os.getcwd()

from main import main
step_tetra = main(geometry='OCC_step_tetra.py', use_OCC=True, use_POD=True, order=1);

OCC_step_tetra.geo
 mesh contains 20231 elements
 solved theta0 problems    
 solving theta1 snapshots


In [ ]:
step_tetra.keys()
Frequencies=Return_Dict['FrequencyArray']
Tensor=Return_Dict['TensorArray']

# determine number of frequences
nfreq, dum = np.shape(Tensors)

MPT = np.empty((3,3),dtype=complex)
Rtil = np.empty((3,3),dtype=complex)
I = np.empty((3,3),dtype=complex)

Angles=np.zeros(nfreq)
Directions=np.zeros((nfreq,3))
#print(np.shape(MPT))
for n in range(nfreq):
    for i in range(3):
        for j in range(3):
            MPT[i,j] = Tensors[n,i*3+j]

    # Set real and imaginary parts
    Rtil = np.real(MPT)
    I = np.imag(MPT)

    # Compute eigen decomposition
    Rtilw, Rtilv = LA.eig(Rtil)
    Iw, Iv = LA.eig(I)

    # We are intrested in the angle between the Rtilv and Iv
#    Rot= Rtilv@Iv
    Rot=Rtilv@np.transpose(Iv)
    Com=Rtil@I -I@Rtil
#    Comw, Comv = LA.eig(Rtil)
#    Rot=Comv

    # Rot = exp (angle * K)
    # Find K and angle by using the matrix exponential. Note that ||K||_2 =1

    # Determine the direction as implied by K
    # Note K is stored as K = [ 0 -kz ky; kz 0 -kx ; -ky kx 0]
    #Knonorm = logm(Rot)
    Knonorm=Com
    Knorm = LA.norm(Knonorm,ord=2)
    K=Knonorm/Knorm
    angle=LA.norm(Knonorm,ord=2)
    kx=K[2,1]
    ky=K[0,2]
    kz=-K[0,1]

    print(K,LA.norm(K,ord=2), np.sqrt(kx**2+ky**2+kz**2))

    Angles[n]=angle
    Directions[n,0]=kx
    Directions[n,1]=ky
    Directions[n,2]=kz

# Show some plots
plt.figure()
plt.plot(Frequencies,Angles)
plt.xscale('log')
plt.xlabel("Frequencies (rad/s)")
plt.ylabel("Angle (rad)")
plt.show()

plt.figure()
plt.plot(Frequencies,Directions[:,0],label='k_x')
plt.plot(Frequencies,Directions[:,1],label='k_y')
plt.plot(Frequencies,Directions[:,2],label='k_z')
plt.xscale('log')
plt.xlabel("Frequencies (rad/s)")
plt.ylabel("Direction")
plt.legend()
plt.show()
